Q1: Edge AI vs Cloud AI — Latency and Privacy
Edge AI performs AI computations directly on local devices (eg, drones, smartphones, sensors), whereas Cloud AI sends data to remote servers for processing. This difference leads to two major advantages for Edge AI:

1. Reduced Latency: Since data doesn't need to travel to the cloud and back, decisions can be made in real time. This is critical for time-sensitive applications like autonomous drones, where milliseconds matter for obstacle avoidance or navigation.
2. Enhanced Privacy: Sensitive data (e.g., video feeds, biometric inputs) stays on the device, minimizing exposure to breaches or unauthorized access during transmission or storage in the cloud.
Real-world example — Autonomous Drones:
- In search-and-rescue missions, drones equipped with Edge AI can analyze terrain, detect humans, and make navigation decisions instantly without relying on cloud connectivity.
- This ensures faster response times and secure handling of sensitive imagery, especially in remote or disaster-struck areas where connectivity is unreliable.

 Q2: Quantum AI vs Classical AI — Optimization Power
Classical AI uses traditional computing methods (binary bits) and algorithms like gradient descent or heuristics to solve optimization problems. These methods work well but struggle with combinatorial complexity and local minima traps in large-scale problems.
Quantum AI, powered by quantum computing, leverages qubits and principles like superposition and entanglement to explore multiple solutions simultaneously. This enables:
- Exponential speedups in solving problems like the traveling salesman, portfolio optimization, or molecular simulations.
- Better global optima discovery in complex landscapes.
Industries poised to benefit most:
- Logistics: Route optimization, warehouse management, and supply chain coordination.
- Finance: Risk modeling, fraud detection, and portfolio optimization.
- Pharmaceuticals: Drug discovery through molecular modeling and protein folding.
- Energy: Grid optimization and predictive maintenance.







In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Define image size and batch size
IMG_HEIGHT = 180
IMG_WIDTH = 180
BATCH_SIZE = 32

print(f"TensorFlow Version: {tf.__version__}")
print(f"Defined Image Height: {IMG_HEIGHT}, Image Width: {IMG_WIDTH}")
print(f"Defined Batch Size: {BATCH_SIZE}")

# Download and load the tf_flowers dataset
# The 'as_supervised=True' argument loads the dataset as (image, label) pairs.
# The 'with_info=True' argument returns a tfds.core.DatasetInfo object.
(train_ds_raw, val_ds_raw), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'], # Split the 'train' portion into 80% train and 20% validation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

num_classes = info.features['label'].num_classes
class_names = info.features['label'].names

print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")
print(f"Number of training samples: {tf.data.experimental.cardinality(train_ds_raw).numpy() * BATCH_SIZE}")
print(f"Number of validation samples: {tf.data.experimental.cardinality(val_ds_raw).numpy() * BATCH_SIZE}")

TensorFlow Version: 2.20.0
Defined Image Height: 180, Image Width: 180
Defined Batch Size: 32
Number of classes: 5
Class names: ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
Number of training samples: 93952
Number of validation samples: 23488


In [4]:
AUTOTUNE = tf.data.AUTOTUNE

# Preprocessing function to resize and normalize images
def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    return image, label

# Apply preprocessing and configure datasets for performance
train_ds = train_ds_raw.map(preprocess_image, num_parallel_calls=AUTOTUNE)
val_ds = val_ds_raw.map(preprocess_image, num_parallel_calls=AUTOTUNE)

# Configure for performance
train_ds = train_ds.cache().shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# Verify the number of samples after splitting and before batching for more accurate count
# tf.data.experimental.cardinality returns the number of elements in the dataset, not batches
num_train_samples = info.splits['train'].num_examples * 0.8 # Approx 80% of total train split
num_val_samples = info.splits['train'].num_examples * 0.2 # Approx 20% of total train split

print(f"Processed training dataset has {len(train_ds)} batches, each with {BATCH_SIZE} samples (except possibly the last one).")
print(f"Processed validation dataset has {len(val_ds)} batches, each with {BATCH_SIZE} samples (except possibly the last one).")
print(f"Approximate number of training samples: {int(num_train_samples)}")
print(f"Approximate number of validation samples: {int(num_val_samples)}")


Processed training dataset has 92 batches, each with 32 samples (except possibly the last one).
Processed validation dataset has 23 batches, each with 32 samples (except possibly the last one).
Approximate number of training samples: 2936
Approximate number of validation samples: 734


In [5]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# 1. Instantiate MobileNetV2 as the base model
# include_top=False removes the classifier layer at the top
# weights='imagenet' loads weights pre-trained on ImageNet
base_model = MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model's layers
base_model.trainable = False

# 2. Create a new classification head
# Use Functional API to build the model
inputs = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = base_model(inputs, training=False) # Apply base model in inference mode
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x) # Add a dropout layer for regularization
outputs = layers.Dense(num_classes, activation='softmax')(x)

# 3. Combine the base model and the new classification head
model = models.Model(inputs, outputs)

# Print model summary to verify the architecture
model.summary()

# 4. Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define the number of training epochs
EPOCHS = 10

# 5. Train the compiled model
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)

print(f"Model trained for {EPOCHS} epochs.")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4460\758348507.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,264,389 (8.64 MB)

 Trainable params: 6,405 (25.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 91s 879ms/step - accuracy: 0.6669 - loss: 0.8738 - val_accuracy: 0.8488 - val_loss: 0.4464
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 694ms/step - accuracy: 0.8345 - loss: 0.4800 - val_accuracy: 0.8706 - val_loss: 0.3735
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 63s 691ms/step - accuracy: 0.8542 - loss: 0.4075 - val_accuracy: 0.8815 - val_loss: 0.3464
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 700ms/step - accuracy: 0.8774 - loss: 0.3478 - val_accuracy: 0.8842 - val_loss: 0.3394
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 693ms/step - accuracy: 0.8934 - loss: 0.3093 - val_accuracy: 0.8869 - val_loss: 0.3278
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 700ms/step - accuracy: 0.9046 - loss: 0.2844 - val_accuracy: 0.8869 - val_loss: 0.3291
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 63s 684ms/step - accuracy: 0.9176 - loss: 0.2578 - val_accuracy: 0.8951 - val_loss: 0.3162
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 64s 701ms/step - accuracy: 0.9149 - loss: 0.2476 - val_accu

In [6]:


# 1. Instantiate a TFLiteConverter object from the trained model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 2. Enable default optimizations, which include dynamic range quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 3. Convert the model to TensorFlow Lite format
tflite_model = converter.convert()

# 4. Save the converted TensorFlow Lite model to a file
with open('flower_classification_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TensorFlow Lite model saved as 'flower_classification_model.tflite'")

INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpi5s5pkrn\assets


INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpi5s5pkrn\assets


Saved artifact at 'C:\Users\ADMINI~1\AppData\Local\Temp\tmpi5s5pkrn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1944050033360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050030480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050030288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050028752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050032016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050028944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050031056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050028176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050031248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1944050033744: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [7]:


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="flower_classification_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite model loaded successfully.")
print("Input details:", input_details)
print("Output details:", output_details)

C:\Users\Administrator\AppData\Roaming\Python\Python313\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite model loaded successfully.
Input details: [{'name': 'serving_default_keras_tensor_154:0', 'index': 0, 'shape': array([  1, 180, 180,   3], dtype=int32), 'shape_signature': array([ -1, 180, 180,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 172, 'shape': array([1, 5], dtype=int32), 'shape_signature': array([-1,  5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [8]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="flower_classification_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("TFLite model loaded successfully.")
print("Input details:", input_details)
print("Output details:", output_details)

# Prepare to evaluate the TFLite model
correct_predictions = 0
total_samples = 0

# Get input and output tensor indexes
input_index = input_details[0]['index']
output_index = output_details[0]['index']

# Iterate through the validation dataset for inference
for images_batch, labels_batch in val_ds:
    for i in range(images_batch.shape[0]):
        img = images_batch[i]
        label = labels_batch[i]

        # TFLite models expect input in a specific format, often float32 and batched.
        # The image is already preprocessed (resized and normalized to [0,1]) and is float32.
        # Add a batch dimension if it's not already there (interpreter expects NCHW or NHWC format).
        input_data = np.expand_dims(img, axis=0).astype(input_details[0]['dtype'])

        # Set the input tensor
        interpreter.set_tensor(input_index, input_data)

        # Invoke the interpreter
        interpreter.invoke()

        # Get the output tensor
        output_data = interpreter.get_tensor(output_index)

        # Get the predicted class (index with highest probability)
        predicted_class = np.argmax(output_data)

        # Compare with true label
        if predicted_class == label.numpy():
            correct_predictions += 1
        total_samples += 1

# Calculate accuracy
accuracy = correct_predictions / total_samples

print(f"\nTotal samples evaluated: {total_samples}")
print(f"Correct predictions: {correct_predictions}")
print(f"TFLite Model Accuracy on Validation Set: {accuracy:.4f}")

TFLite model loaded successfully.
Input details: [{'name': 'serving_default_keras_tensor_154:0', 'index': 0, 'shape': array([  1, 180, 180,   3], dtype=int32), 'shape_signature': array([ -1, 180, 180,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'StatefulPartitionedCall_1:0', 'index': 172, 'shape': array([1, 5], dtype=int32), 'shape_signature': array([-1,  5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

Total samples evaluated: 734
Correct predictions: 653
TFLite Model Accuracy on Validation Set: 0.8896


Explain Benefits of Edge AI
 Benefits of Edge AI for Real-time Applications:

1.  Reduced Latency: By performing inference directly on the device (e.g., a smartphone, drone, or industrial sensor) rather than sending data to a remote cloud server, Edge AI drastically reduces the time it takes to get a prediction. For real-time applications like autonomous driving, drone navigation (as mentioned in Q1), or real-time object detection in surveillance, milliseconds matter. Our TFLite model, being lightweight and optimized, can run quickly on edge devices, enabling immediate responses.

2.  Enhanced Privacy and Security: Sensitive data, such as images from security cameras or personal biometric information, can be processed locally on the device. This minimizes the need to transmit data over networks, reducing the risk of data breaches or unauthorized access during transit or storage in the cloud. For a flower classification model, while less sensitive, this principle scales up significantly for applications dealing with private user data.

3.  Lower Bandwidth and Connectivity Costs: Edge AI reduces the amount of data that needs to be sent to the cloud. This saves bandwidth, which can be critical in areas with limited or expensive network access. It also allows applications to function reliably even when offline or in environments with intermittent connectivity, such as remote agricultural fields for crop monitoring or disaster zones for search and rescue.

4.  Increased Reliability: Cloud-dependent AI systems can fail if the internet connection is lost or the cloud service experiences downtime. Edge AI systems can operate independently of constant network connectivity, ensuring continuous operation, which is vital for critical real-time systems like medical devices or industrial control systems.

5.  Lower Power Consumption on Data Transfer: While running AI models on edge devices consumes power, the overall system can be more energy-efficient by avoiding constant data transmission to and from the cloud, especially for devices powered by batteries.

6.  Scalability: Edge AI can enable more devices to participate in an AI ecosystem without overwhelming central cloud servers with massive amounts of raw data. Each edge device handles its own data processing, allowing for distributed and scalable AI deployments.

TASK2:AI DRIVEN IOT CONCEPT
key sensors needed are;

Soil Moisture Sensor: Measures water content in the soil to optimize irrigation.

Soil Temperature Sensor: Monitors root-zone temperature, crucial for seed germination and microbial activity.

Soil NPK Sensor: Measures the levels of essential macronutrients: Nitrogen (N), Phosphorus (P), and Potassium (K).

Soil pH Sensor: Monitors soil acidity/alkalinity, which affects nutrient availability.

Leaf Wetness Sensor: Detects moisture on leaf surfaces, helping predict fungal disease outbreaks.

Water pH Sensor: (For irrigation systems) Monitors the pH of the water being used.

Water Flow Sensor: Measures the amount of water used for irrigation.



Core AI Model: Hybrid Ensemble (LSTM + XGBoost)

a) Temporal Feature Learning with LSTM (Long Short-Term Memory Network)

Purpose: To model time-dependent patterns in the sensor data.

Input: Sequential, time-series data from sensors (soil moisture, temperature, humidity, etc.) over the entire growing season.


b) Static & Spatial Feature Integration with XGBoost (Extreme Gradient Boosting)

Purpose: To integrate non-sequential, static, and spatial features.

Input:

Static Features: Soil type, seed variety, historical field yield data.

Spatial Features: Data from multispectral imagery (e.g., NDVI - Normalized Difference Vegetation Index) at different growth stages.

Aggregated LSTM Features: The learned features from the LSTM model's hidden layers can be fed into XGBoost as powerful summary inputs.

+--------------------------------------------------------------------+
|                  AI-DRIVEN SMART AGRICULTURE SYSTEM                |
+--------------------------------------------------------------------+

+----------------+      +-----------------+      +-----------------+
|                |      |                 |      |                 |
|  SENSOR LAYER  |----->|   EDGE/GATEWAY  |----->|   CLOUD LAYER   |
|                |      |     LAYER       |      |                 |
+----------------+      +-----------------+      +-----------------+
     |  |  |                    |                        |
     |  |  |                    |                        |
     v  v  v                    v                        v
+-----------+             +-------------+          +-------------+
|Soil       | Wireless    |Data         |   Internet  |Data        |
|Moisture   |-----------> |Aggregation  |----------> |Storage     |
+-----------+   (LoRaWAN) +-------------+            +-------------+
+-----------+                               (Data Lake)  |
|Temperature|                                            |
|& Humidity |                                            |
+-----------+                                            |
+-----------+                                            |
|NPK Sensor |                                            v
+-----------+                                    +-------------+
+-----------+                                    |AI PROCESSING|
|Multispectral|                                   +-------------+
|Camera (Drone)|                                  |             |
+--------------+                                  | LSTM Model  |
                                                  |             |
                                                  +-------------+
                                                          |
                                                          v
+----------------+      +-----------------+      +-------------+
|                |      |                 |      | PREDICTIONS &|
|  ACTION LAYER  |<-----|   DASHBOARD     |<-----|   INSIGHTS   |
|                |      |                 |      |             |
+----------------+      +-----------------+      +-------------+
     ^                         ^                        ^
     |                         |                        |
+-----------+             +-------------+          +-------------+
|Smart      |             |Mobile/Web   |          |Yield        |
|Valves     | Control     |Interface    |          |Prediction   |
+-----------+  Signals    +-------------+          +-------------+
+-----------+             |Alerts &     |          |Disease Risk |
|Fertilizer |             |Notifications|          +-------------+
|Dispenser  |             +-------------+          |Water Stress |
+-----------+                                      +-------------+

